# WorldMet Meteorological Data

The whole curation process have been performed using the `src/` scripts and this notebook only show examples of the process for monitoring sites in Madrid capital.

In [6]:
setwd("AirQualityCOVID/")

In [7]:
# Loading
suppressMessages(library(worldmet))
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))
suppressMessages(library(openair))

In [8]:
study.prd <- c(ymd_hms("2013-01-01 00:00:00"), ymd_hms("2020-12-31 00:00:00"))

period <- as.integer(interval(floor_date(study.prd[1], unit="day"),
                    floor_date(study.prd[2], unit="day")
                   ) / (3600*24)) + 2

## Air Quality sites info

In [11]:
sites.AQ <- read.csv("data/curation/checked_AQ.csv",
                     stringsAsFactor=TRUE) %>%
                filter(Municipio == "Madrid") %>%
                mutate(site=as.character(site))

head(sites.AQ)

,site,variable,site_name,latitude,longitude,elevation,country,site_type,site_area,date_start,date_end,Municipio,Poblaci..n,Estaci..n.tr..fico
,<chr>,<fct>,<fct>,<dbl>,<dbl>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>
1,es0118a,no,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
2,es0118a,no2,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
3,es0118a,o3,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
4,es0118a,pm10,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
5,es0118a,pm2.5,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
6,es0120a,no,RAM<93>N Y CAJAL,40.45167,-3.677222,708,spain,traffic,urban,2002-01-01 00:00:00,2021-05-10 19:00:00,Madrid,3266126,RAM<93>NYCAJAL


In [14]:
sites.Mto <- data.frame()
sites.lv <- levels(as.factor(sites.AQ$site))

for (st in sites.lv) {
    mto <- getMeta(lat = sites.AQ[sites.AQ$site == st, ]$latitude[1], 
                   lon = sites.AQ[sites.AQ$site == st, ]$longitude[1], 
                   end.year = "current",
                   n = 3, returnMap = F)

    for (cd in mto[order(mto$dist), ]$code) {
        fileName <- paste("data/curation/NOAA-ISD/", cd, ".csv", sep="")
        
        if (file.exists(fileName)) {
            mto[mto$code == cd, "siteAQ"] <- st
            sites.Mto <- rbind(sites.Mto, mto[mto$code == cd, ])            
            
            break
        } else {
            data <- importNOAA(code = cd,
                              year = 2013:2020,
                              hourly = TRUE,
                              n.cores = 6
                             ) %>%
                        select("date", "code", "ws", "wd", "atmos_pres") %>%
                        timeAverage(avg.time = "day", type="code")

            if(sum(colSums(!is.na(data))/ period > 0.8) == ncol(data)) {
                mto[mto$code == cd, "siteAQ"] <- st
                sites.Mto <- rbind(sites.Mto, mto[mto$code == cd, ])

                break
            }
        }
    }
}

[1] "es0118a"
[1] "The following sites / years are missing:"
          code year date station
1 082220-99999 2013 <NA>    <NA>
2 082220-99999 2014 <NA>    <NA>
3 082220-99999 2015 <NA>    <NA>
4 082220-99999 2016 <NA>    <NA>
[1] "The following sites / years are missing:"
          code year date station
1 082200-99999 2013 <NA>    <NA>
2 082200-99999 2014 <NA>    <NA>
3 082200-99999 2015 <NA>    <NA>
4 082200-99999 2016 <NA>    <NA>
[1] "es0120a"
[1] "The following sites / years are missing:"
          code year date station
1 082200-99999 2013 <NA>    <NA>
2 082200-99999 2014 <NA>    <NA>
3 082200-99999 2015 <NA>    <NA>
4 082200-99999 2016 <NA>    <NA>
[1] "The following sites / years are missing:"
          code year date station
1 082220-99999 2013 <NA>    <NA>
2 082220-99999 2014 <NA>    <NA>
3 082220-99999 2015 <NA>    <NA>
4 082220-99999 2016 <NA>    <NA>
[1] "es1426a"
[1] "The following sites / years are missing:"
          code year date station
1 082220-99999 2013 <NA>    <N